<a href="https://colab.research.google.com/github/ommore1212/Credit-Card-Fraud-Detection/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the Dependencies




In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Data Collection and Data Processing

In [3]:


file_path = "creditcard.csv"
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "mlg-ulb/creditcardfraud",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

<ipython-input-3-ced36c37f90b>:2: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


100%|██████████| 66.0M/66.0M [00:00<00:00, 144MB/s]

Extracting zip of creditcard.csv...


In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df.shape

(284807, 31)

In [8]:
df['Class'].value_counts()

,count
Class,
0,284315
1,492


In [9]:
df.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [10]:
df['Class'].value_counts()

,count
Class,
0,284315
1,492


# This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [27]:
  # separating the data for analysis
legit = df[df.Class == 0]
fraud =df[df.Class == 1]

In [28]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [29]:
legit.Amount.describe()


,Amount
count,284315.000000
mean,88.291022
std,250.105092
min,0.000000
25%,5.650000
50%,22.000000
75%,77.050000
max,25691.160000


In [30]:
fraud.Amount.describe()


,Amount
count,492.000000
mean,122.211321
std,256.683288
min,0.000000
25%,1.000000
50%,9.250000
75%,105.890000
max,2125.870000


In [31]:
# compare the values for both transactions
df.groupby('Class').mean()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


# Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 492

In [32]:
legit_sample = legit.sample(n=492)

# Concatenating two DataFrames

In [33]:
new_dataset = pd.concat([legit_sample,fraud],axis = 0 )

In [34]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
252213,155700.0,-1.881447,1.889911,1.325540,4.649666,-0.499826,1.048604,-0.514981,0.042839,-2.025488,...,1.023289,0.551002,-0.364755,0.053245,0.487421,0.689015,-0.254900,-0.090368,67.18,0
44129,41807.0,1.101261,-0.584512,0.766545,0.750180,-0.973317,0.136889,-0.711839,0.244446,1.347016,...,-0.131601,-0.459573,-0.106215,-0.482392,0.257933,0.369563,-0.013391,0.025872,79.99,0
11433,19895.0,1.383524,-0.344375,0.809982,-0.183156,-1.106732,-0.945422,-0.578919,-0.323454,0.678106,...,-0.142525,-0.014423,0.011979,0.702952,0.524023,-0.305569,-0.005312,0.011266,5.00,0
85457,60781.0,1.157730,-1.488516,1.345698,-0.133224,-1.945988,0.550598,-1.565042,0.332068,1.075891,...,0.096466,0.440162,-0.228254,-0.093221,0.505720,-0.018223,0.064854,0.034721,97.35,0
194118,130449.0,-0.272199,1.041260,0.070025,-0.229263,0.745998,-1.207435,1.249685,-0.310373,-0.390011,...,0.303380,0.915466,-0.331488,-0.083228,0.099752,-0.059523,0.145426,0.182278,38.01,0


In [35]:
new_dataset['Class'].value_counts()

,count
Class,
0,492
1,492


In [36]:
new_dataset.groupby('Class').mean()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,96078.142276,-0.058714,-0.089611,-0.013388,0.087621,0.030697,0.078570,0.013027,0.036576,0.091370,...,-0.040108,-0.049907,-0.048100,-0.026481,0.004632,-0.028752,-0.021584,0.022462,0.008828,96.052541
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


# Splitting the data into Features & Targets

In [37]:
x = new_dataset.drop(columns='Class', axis=1)
y = new_dataset['Class']

# Split the data into Training data & Testing Data

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2)

In [40]:
print(x.shape, x_train.shape, x_test.shape)

(984, 30) (787, 30) (197, 30)


# Model Training

Logistic Regression

In [41]:
model = LogisticRegression()

In [43]:
model.fit(x_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

#Model Evaluation

Accuracy Score

In [44]:
# accuracy on training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [45]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9466327827191868


In [46]:
# accuracy on test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [47]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.934010152284264
